In [45]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier


In [46]:
# Load the dataset
data = pd.read_csv('dataset_phishing.csv')
y = data['status'].apply(lambda x: 1 if x == 'phishing' else 0).values

In [47]:
all_features = data.columns.tolist()
all_features.remove('status') 
# tf-idf embedding for url
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 5), max_features=1000)
X_tfidf = vectorizer.fit_transform(data['url']).toarray()
tfidf_feature_names = vectorizer.get_feature_names_out()

# extract numerical features
numerical_features = all_features.copy()
numerical_features.remove('url')  
X_numerical = data[numerical_features].values

# combine url with rest featrues and split
X_combined = np.hstack((X_tfidf, X_numerical))
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.1, random_state=42)

# use random forest to rate feature importance
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)
feature_importances = rf.feature_importances_

# find top 20 important features in url
url_feature_importances = feature_importances[:len(tfidf_feature_names)]
url_features_with_importance = list(zip(tfidf_feature_names, url_feature_importances))
top_ten_url_features = sorted(url_features_with_importance, key=lambda x: x[1], reverse=True)[:20]
# print("Top 20 URL-related features:")
# for feature, importance in top_ten_url_features:
#     print(f"Feature: {feature}, Importance: {importance}")

# find top 5 important features except url
remaining_feature_importances = feature_importances[len(tfidf_feature_names):]
remaining_feature_names = numerical_features
remaining_features_with_importance = list(zip(remaining_feature_names, remaining_feature_importances))
top_five_features = sorted(remaining_features_with_importance, key=lambda x: x[1], reverse=True)[:5]
print("Top five features (excluding url-related features):")
for feature, importance in top_five_features:
    print(f"Feature: {feature}, Importance: {importance}")


Top five features (excluding url-related features):
Feature: google_index, Importance: 0.08119731915700858
Feature: page_rank, Importance: 0.054764258448125436
Feature: nb_hyperlinks, Importance: 0.04117338134687459
Feature: web_traffic, Importance: 0.03248001035621982
Feature: domain_age, Importance: 0.020893432999659213


In [48]:
# Feature and label extraction, including additional numerical features
selected_features = [ 'length_url', 'nb_dots', 'https_token', 'nb_subdomains', 'google_index', 'nb_hyperlinks']  # Add other selected feature column names
# selected_features = [item[0] for item in top_five_features]
X_numerical = data[selected_features].values  # Extract numerical features

# Character n-gram TF-IDF vectorization for the 'url' feature
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 5), max_features=1000)
X_tfidf = vectorizer.fit_transform(data['url']).toarray()

# top_features_names = [feature[0] for feature in top_ten_url_features]
# feature_indices = [list(tfidf_feature_names).index(feature) for feature in top_features_names]
# feature_values = X_tfidf[:, feature_indices]

# Combine TF-IDF features with numerical features
X_combined = np.hstack((X_tfidf, X_numerical))
# X_combined = np.hstack((feature_values, X_numerical))

# Fit the scaler only on the training data and transform both sets
X_train, X_temp, y_train, y_temp = train_test_split(X_combined, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# DataLoader
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=64, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=64, shuffle=False)

In [49]:
class Attention(nn.Module):
    def __init__(self, feature_dim):
        super(Attention, self).__init__()
        self.feature_dim = feature_dim
        self.proj = nn.Linear(feature_dim, 64)
        self.out = nn.Linear(64, feature_dim)  # Ensure output dimension matches feature_dim

    def forward(self, x):
        eij = self.proj(x)
        eij = torch.tanh(eij)
        eij = self.out(eij)
        a = torch.softmax(eij, dim=1)
        weighted_input = x * a
        return weighted_input  # Return the weighted input for further processing

class AdvancedURLNetWithAttention(nn.Module):
    def __init__(self, num_features):
        super(AdvancedURLNetWithAttention, self).__init__()
        # Increased the complexity of the first layer and added a second attention layer
        self.fc1 = nn.Linear(num_features, 1024)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.3)  # Reduced dropout
        self.attention1 = Attention(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.3)
        self.attention2 = Attention(512)  # New attention layer
        self.fc3 = nn.Linear(512, 128)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.3)
        self.fc4 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.attention1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.attention2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.dropout3(x)
        x = self.fc4(x)
        x = self.sigmoid(x)
        return x

In [50]:

# Define the model, loss function, and optimizer
model = AdvancedURLNetWithAttention(X_train_tensor.shape[1])  # assuming this class is defined correctly
criterion = nn.BCELoss()  # Binary cross-entropy loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

In [51]:
# Check CUDA availability
cuda_available = torch.cuda.is_available()

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=10, patience=3):
    if cuda_available:
        model = model.cuda()
    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(epochs):
        model.train()
        for inputs, labels in train_loader:
            if cuda_available:
                inputs, labels = inputs.cuda(), labels.cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()

        scheduler.step()

        model.eval()
        with torch.no_grad():
            val_loss = 0
            for val_inputs, val_labels in val_loader:
                if cuda_available:
                    val_inputs, val_labels = val_inputs.cuda(), val_labels.cuda()
                val_outputs = model(val_inputs)
                val_loss += criterion(val_outputs.squeeze(), val_labels).item()
            val_loss /= len(val_loader)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print(f'Early stopping at epoch {epoch+1}')
                break

        if epoch % 5 == 0 or epoch == epochs - 1:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}, Val Loss: {val_loss}")

    return 


# Evaluation function
def evaluate_model(model, test_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            if cuda_available:
                inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            predicted = (outputs.squeeze() > 0.5).float()
            all_preds.extend(predicted.cpu().numpy())  # Move predictions back to CPU
            all_labels.extend(labels.cpu().numpy())    # Move labels back to CPU
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}")



In [52]:
# Train and evaluate the model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=50, patience=5)

Epoch 1/50, Loss: 0.3406546711921692, Val Loss: 0.3018457219004631
Epoch 6/50, Loss: 0.06166786327958107, Val Loss: 0.1742662334193786
Early stopping at epoch 10


In [53]:
model.to('cuda')
evaluate_model(model, test_loader)

Accuracy: 0.9326334208223972, Precision: 0.9309734513274336, Recall: 0.9326241134751773, F1-Score: 0.9317980513728963


In [58]:
model.to('cpu')

def transform_and_predict(features_list, vectorizer, scaler, model):
    # Separate the URL from the numerical features
    url, numerical_features = features_list[0], np.array(features_list[1:])

    # Transform the URL using the pre-fitted TF-IDF vectorizer
    tfidf_features = vectorizer.transform([url]).toarray()

    # Ensure numerical_features is a 2D array with a single sample
    numerical_features = numerical_features.reshape(1, -1)

    # Combine TF-IDF features with other numerical features
    combined_features = np.hstack((tfidf_features, numerical_features))

    # Standardize the features using the pre-fitted scaler
    scaled_features = scaler.transform(combined_features)  # No need for additional brackets now

    # Convert to PyTorch tensor
    features_tensor = torch.tensor(scaled_features, dtype=torch.float32)

    # Predict using the model
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        prediction = model(features_tensor)
        predicted_class = (prediction.squeeze() > 0.5).float()
        return predicted_class.item()  # Return the prediction as a Python scalar

# Example usage for a single URL and its numerical features
single_features = ['https://google.com', 18, 1, 0, 10000, 1, 18]   # URL followed by its numerical features
prediction = transform_and_predict(single_features, vectorizer, scaler, model)
print("Phishing" if prediction == 1 else "Legitimate")

Legitimate


In [55]:
import joblib

# Save the PyTorch model's state dictionary
torch.save(model.state_dict(), 'model_state_dict.pth')

# Save the fitted TF-IDF vectorizer
joblib.dump(vectorizer, 'tfidf_vectorizer.joblib')

# Save the fitted scaler
joblib.dump(scaler, 'standard_scaler.joblib')

['standard_scaler.joblib']